In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [7]:
data = pd.read_csv('dataset/wrangled_data.csv',
                       encoding='utf-8')

In [8]:
data['text'] = data['text'].str.lower()

In [9]:
data

,text,class
0,sci-news.com is committed to protecting and re...,Other
1,sci-news.com is committed to protecting and re...,Other
2,please note that these terms are subject to ch...,Policy Change
3,sci-news.com may collect and process nformatio...,First Party Collection/Use
4,sci-news.com may collect and process nformatio...,First Party Collection/Use
...,...,...
19353,do not ersonal information children,First Party Collection/Use
19354,if you have a question regarding any of the ab...,Other
19355,information@mohegansun.com,Other
19356,"1 mohegan sun boulevard uncasville, ct 06382 f...",Other


In [10]:
dict = {}

def find_all_classes(text):
    try:
        dict[text]
    except:
        dict[text] = len(dict)
           
#data['class'].apply(find_all_classes)

dict

{'Other': 0,
 'Policy Change': 1,
 'First Party Collection/Use': 2,
 'Data Retention': 3,
 'International and Specific Audiences': 4,
 'Third Party Sharing/Collection': 5,
 'User Choice/Control': 6,
 'User Access, Edit and Deletion': 7,
 'Data Security': 8,
 'Do Not Track': 9}

In [11]:
def replace_classes_with_numbers(text):
    return dict[text]

data['class'] =data['class'].apply(replace_classes_with_numbers)

data

,text,class
0,sci-news.com is committed to protecting and re...,0
1,sci-news.com is committed to protecting and re...,0
2,please note that these terms are subject to ch...,1
3,sci-news.com may collect and process nformatio...,2
4,sci-news.com may collect and process nformatio...,2
...,...,...
19353,do not ersonal information children,2
19354,if you have a question regarding any of the ab...,0
19355,information@mohegansun.com,0
19356,"1 mohegan sun boulevard uncasville, ct 06382 f...",0


In [12]:
stop_words = stopwords.words('english')
stop_words.append('may') #???
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
print(data['text'])

0        sci-news.com committed protecting respecting p...
1        sci-news.com committed protecting respecting p...
2        please note terms subject change, changes incl...
3        sci-news.com collect process nformation provid...
4        sci-news.com collect process nformation provid...
                               ...                        
19353                         ersonal information children
19354    question regarding information, please contact...
19355                           information@mohegansun.com
19356    1 mohegan sun boulevard uncasville, ct 06382 g...
19357    1 mohegan sun boulevard uncasville, ct 06382 g...
Name: text, Length: 19358, dtype: object


In [13]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

data['text'] = data['text'].apply(lemmatize_text)

In [15]:
data.to_csv('dataset/train_preprocessed.csv')

In [14]:
print(data['class'].value_counts())
data['class'].value_counts().plot(kind='bar')

2    7866
5    4712
0    2398
6    1582
8     734
7     669
4     560
1     438
3     334
9      65
Name: class, dtype: int64
